# 1. Basic Set Up

In [1]:
#load modules
import numpy as np #version 1.18.1
import pandas as pd #version 1.0.1
from scipy.optimize import nnls #version 1.4.1

In [2]:
#load RSL rise magnitude at each site
#------------------Empirical Senario------------------
Emp_mag = pd.read_excel('RSL magnitude/Lin et al., Sup data.xlsx',sheet_name=1) 
Tahiti_emp_dis = Emp_mag['Tahiti']
Tahiti_emp_std = np.std(Tahiti_emp_dis)
Barbados_emp_dis =  Emp_mag['Barbados']
Barbados_emp_std = np.std(Barbados_emp_dis)
Sunda_emp_dis =  Emp_mag['Sunda Shelf']
Sunda_emp_std = np.std(Sunda_emp_dis)
HYD_emp_dis =  Emp_mag['Hydrographer\'s Passage (HYD)']
HYD_emp_std = np.std(HYD_emp_dis)
NOG_emp_dis =  Emp_mag['Noggin Pass (NOG)']
NOG_emp_std = np.std(NOG_emp_dis)
Scot_emp_dis =  Emp_mag['Northwest Scotland']
Scot_emp_std = np.std(Scot_emp_dis)
total_emp_std = np.array([Tahiti_emp_std,Barbados_emp_std,Sunda_emp_std,HYD_emp_std,NOG_emp_std,
                        Scot_emp_std]) 
#------------------Uniform Senario------------------
Uni_mag = pd.read_excel('RSL magnitude/Lin et al., Sup data.xlsx',sheet_name=2) #Uniform Senario
Tahiti_uni_dis = Uni_mag['Tahiti']
Tahiti_uni_std = np.std(Tahiti_uni_dis)
Barbados_uni_dis =  Uni_mag['Barbados']
Barbados_uni_std = np.std(Barbados_uni_dis)
Sunda_uni_dis =  Uni_mag['Sunda Shelf']
Sunda_uni_std = np.std(Sunda_uni_dis)
HYD_uni_dis =  Uni_mag['Hydrographer\'s Passage (HYD)']
HYD_uni_std = np.std(HYD_uni_dis)
NOG_uni_dis =  Uni_mag['Noggin Pass (NOG)']
NOG_uni_std = np.std(NOG_uni_dis)
Scot_uni_dis =  Uni_mag['Northwest Scotland']
Scot_uni_std = np.std(Scot_uni_dis)
total_uni_std = np.array([Tahiti_uni_std,Barbados_uni_std,Sunda_uni_std,HYD_uni_std,NOG_uni_std,
                        Scot_uni_std])
#load sea-level fingerprint matrix
#each row corresponds to each sea-level site and each column corresponds to each ice sheet
A =  np.array(pd.read_csv('RSL magnitude/fingerprint.csv',index_col=0))


In [3]:
def find_maximum_prob(values,percent):
    '''This function is used to analytically find the smallest confidence interval  
    range from a data ensemble 
    values:  data samples
    percent: probablity range to find (e.g., 95 is 95% CI)'''
    
    v_max,v_min = np.max(values),np.min(values)
    ranges = []
    for i in np.linspace(0,100-percent,1000):
        _b,_e = np.percentile(values,[i,i+percent])
        ranges.append([_b,_e,np.abs(_b-_e),i,i+percent])
    ranges = np.array(ranges)
    min_index = np.argmin(ranges[:,2])
    
    return ranges[min_index]

# 2. MWP-1A Sources Inversion

All loaded 20,000 MWP-1A RSL rise magnitudes samples represent the elastic induced relative sea-level rise at each location, which can be used to invert the MWP-1A sources following the equation:



$$ \Delta {RSL}_{Elastic}(\varphi)  = {ESL}_{NAIS}  \times F_{NAIS}(\varphi) + {ESL}_{AIS} \times F_{AIS}(\varphi) +{ESL}_{SIS}\times F_{SIS}(\varphi)$$

where $\Delta {RSL}_{Elastic}(\varphi)$ represent the instantaneous response of the solid Earth and sea surface to an influx of meltwater at location $\varphi$; three $ESL$ terms represent eustatic sea-level (ESL) contributions from the NAIS, AIS, and SIS ice sheets and the $F(\varphi)$ terms denote ice-sheet-specific, site-specific sea-level fingerprint values.


By optimising this equation above using the weighted non-negative least sqaure approch based on Lawson-Hanson algorithm (from a [scipy](https://docs.scipy.org/doc/scipy/reference/index.html) function [nnls](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.nnls.html)), three optimised $ESL$ paramters in the equation above can be obtained. 

### Empirical senario for coral records

In [4]:
emp_result = []
while len(emp_result)<20000:
    i = np.random.randint(0,20000,1)[0]
    obs = np.array([Tahiti_emp_dis.iloc[i],Barbados_emp_dis.iloc[i],Sunda_emp_dis.iloc[i],HYD_emp_dis.iloc[i],
                   NOG_emp_dis.iloc[i],Scot_emp_dis.iloc[i]]) #compile all sample together
    opti = list(nnls(np.sqrt(1/np.array(total_emp_std)**2)[:,None] * A,np.sqrt(1/np.array(total_emp_std)**2)*obs)[0]) #non-negative least square
    emp_result.append(opti) #append results from this sample
    
emp_result= np.array(emp_result)
print('----------------MWP1A Magnitude & Sources with Empirical Distribution -')
print('----------------Mean Result (95% confidence interval)----------------')
print('NAIS: {0:5.3f} [{1:5.3f},{2:5.3f}]'.format(np.mean(emp_result[:,0]),*find_maximum_prob(emp_result[:,0],95)[:2]))
print('AIS: {0:5.3f} [{1:5.3f},{2:5.3f}]'.format(np.mean(emp_result[:,1]),*find_maximum_prob(emp_result[:,1],95)[:2]))
print('SIS:   {0:5.3f} [{1:5.3f},{2:5.3f}]'.format(np.mean(emp_result[:,2]),*find_maximum_prob(emp_result[:,2],95)[:2]))
print('Total MWP-1A Magnitude:   {0:5.3f} [{1:5.3f},{2:5.3f}]'.format(np.mean(np.sum(emp_result,axis=1)),*find_maximum_prob(np.sum(emp_result,axis=1),95)[:2]))

----------------MWP1A Magnitude & Sources with Empirical Distribution -
----------------Mean Result (95% confidence interval)----------------
NAIS: 12.280 [0.000,18.565]
AIS: 2.532 [0.000,10.692]
SIS:   3.490 [0.000,6.565]
Total MWP-1A Magnitude:   18.302 [14.485,22.068]


### Uniform senario coral records

In [5]:
uni_result = []
while len(uni_result)<20000:
    i = np.random.randint(0,20000,1)[0]
    obs = np.array([Tahiti_uni_dis.iloc[i],Barbados_uni_dis.iloc[i],Sunda_uni_dis.iloc[i],HYD_uni_dis.iloc[i],
                   NOG_uni_dis.iloc[i],Scot_uni_dis.iloc[i]]) #compile all sample together
    opti = list(nnls(np.sqrt(1/np.array(total_uni_std)**2)[:,None] * A,np.sqrt(1/np.array(total_uni_std)**2)*obs)[0]) #non-negative least square
    uni_result.append(opti)
    
uni_result= np.array(uni_result)
print('----------------MWP1A Magnitude & Sources with Uniform Distribution -')
print('----------------Mean Result (95% confidence interval)----------------')
print('NAIS: {0:5.3f} [{1:5.3f},{2:5.3f}]'.format(np.mean(uni_result[:,0]),*find_maximum_prob(uni_result[:,0],95)[:2]))
print('AIS: {0:5.3f} [{1:5.3f},{2:5.3f}]'.format(np.mean(uni_result[:,1]),*find_maximum_prob(uni_result[:,1],95)[:2]))
print('SIS:   {0:5.3f} [{1:5.3f},{2:5.3f}]'.format(np.mean(uni_result[:,2]),*find_maximum_prob(uni_result[:,2],95)[:2]))
print('Total MWP-1A Magnitude:   {0:5.3f} [{1:5.3f},{2:5.3f}]'.format(np.mean(np.sum(uni_result,axis=1)),*find_maximum_prob(np.sum(uni_result,axis=1),95)[:2]))

----------------MWP1A Magnitude & Sources with Uniform Distribution -
----------------Mean Result (95% confidence interval)----------------
NAIS: 12.428 [3.512,19.609]
AIS: 2.164 [0.000,8.692]
SIS:   3.124 [0.000,5.953]
Total MWP-1A Magnitude:   17.717 [14.902,20.600]


# 3. Jackknife Resampling

### Empirical senario for coral records

In [6]:
jack_knife_result =np.zeros([60000,3])
site_name = ['Tahiti',"Barbados",'Sunda Shelf','Hydrographer','Noggin Pass',
            'Scotland']
for jack in range(6):
    
    all_emp_result = [Tahiti_emp_dis,Barbados_emp_dis,Sunda_emp_dis,NOG_emp_dis,
                           HYD_emp_dis,Scot_emp_dis]
    total_emp_std = [Tahiti_emp_std,Barbados_emp_std,Sunda_emp_std,HYD_emp_std,NOG_emp_std,
                         Scot_emp_std]
    A_index = ~(np.arange(0,6)==jack)
    A_jack = A[A_index]

    del all_emp_result[jack] #remove one site's observation
    del total_emp_std[jack] #remove one site's standard deviation 
    emp_test_result = []
    all_emp_result = np.array(all_emp_result)
    total_emp_std = np.array(total_emp_std)
    while len(emp_test_result)<10000:
        
        i = np.random.randint(0,20000,1)[0]
        obs = np.array([Tahiti_emp_dis.iloc[i],Barbados_emp_dis.iloc[i],Sunda_emp_dis.iloc[i],HYD_emp_dis.iloc[i],
                       NOG_emp_dis.iloc[i],Scot_emp_dis.iloc[i]])
        obs = obs[A_index] 
        opti = list(nnls(np.sqrt(1/np.array(total_emp_std)**2)[:,None] * A_jack,np.sqrt(1/np.array(total_emp_std)**2)*obs)[0])
        emp_test_result.append(opti)
       
    
    emp_test_result= np.array(emp_test_result)
    jack_knife_result[jack*10000:(jack+1)*10000,:]=emp_test_result
    print('----------------Without {:} -------------------------------------'.format(site_name[jack]))
    print('----------------MWP1A Magnitude & Sources with Empirical Distribution -----------')
    print('----------------Mean Result [95% confidence interval] (1 std)-----------------')
    print('North American Ice Sheet: {0:5.3f} [{1:5.3f},{2:5.3f}]  ({3:5.3f})'.format(np.mean(emp_test_result[:,0]),find_maximum_prob(emp_test_result[:,0],95)[0],find_maximum_prob(emp_test_result[:,0],95)[1],
                                                                                   np.std(emp_test_result[:,0])))
    print('West Antarctic Ice Sheet: {0:5.3f} [{1:5.3f},{2:5.3f}] ({3:5.3f})'.format(np.mean(emp_test_result[:,1]),find_maximum_prob(emp_test_result[:,1],95)[0],find_maximum_prob(emp_test_result[:,1],95)[1],
                                                                                  np.std(emp_test_result[:,1])))
    print('Scandinavian Ice Sheet:   {0:5.3f} [{1:5.3f},{2:5.3f}]  ({3:5.3f})'.format(np.mean(emp_test_result[:,2]),find_maximum_prob(emp_test_result[:,2],95)[0],find_maximum_prob(emp_test_result[:,2],95)[1],
                                                                                  np.std(emp_test_result[:,2])))
    print('----------------------------------------------------------------------')

print('----------------Overall Jackknife/Original Results --------------------')
print('NAIS: {0:5.3f}, {1:5.3f}  Bias: {2:5.3f} Bias Corrected: {3:5.3f} m'.format(np.mean(jack_knife_result[:,0]),np.mean(emp_result[:,0]),
                                                                           np.mean(jack_knife_result[:,0])-np.mean(emp_result[:,0]),
                                                                            np.mean(emp_result[:,0])-(np.mean(jack_knife_result[:,0])-np.mean(emp_result[:,0]))))
print('AIS: {0:5.3f}, {1:5.3f}  Bias: {2:5.3f} Bias Corrected: {3:5.3f} m'.format(np.mean(jack_knife_result[:,1]),np.mean(emp_result[:,1]),
                                                                           np.mean(jack_knife_result[:,1])-np.mean(emp_result[:,1]),
                                                                           np.mean(emp_result[:,1])-(np.mean(jack_knife_result[:,1])-np.mean(emp_result[:,1]))))
print('SIS:   {0:5.3f}, {1:5.3f}  Bias: {2:5.3f} Bias Corrected: {3:5.3f} m'.format(np.mean(jack_knife_result[:,2]),np.mean(emp_result[:,2]),
                                                                          np.mean(jack_knife_result[:,2])-np.mean(emp_result[:,2]),
                                                                        np.mean(emp_result[:,2])-( np.mean(jack_knife_result[:,2])-np.mean(emp_result[:,2]))))
print('----------------------------------------------------------------------')

----------------Without Tahiti -------------------------------------
----------------MWP1A Magnitude & Sources with Empirical Distribution -----------
----------------Mean Result [95% confidence interval] (1 std)-----------------
North American Ice Sheet: 11.395 [0.000,19.050]  (6.207)
West Antarctic Ice Sheet: 3.058 [0.000,11.904] (4.397)
Scandinavian Ice Sheet:   3.479 [0.000,6.568]  (2.029)
----------------------------------------------------------------------
----------------Without Barbados -------------------------------------
----------------MWP1A Magnitude & Sources with Empirical Distribution -----------
----------------Mean Result [95% confidence interval] (1 std)-----------------
North American Ice Sheet: 7.508 [0.000,17.911]  (7.232)
West Antarctic Ice Sheet: 6.912 [0.000,15.973] (6.288)
Scandinavian Ice Sheet:   5.156 [0.000,10.001]  (2.932)
----------------------------------------------------------------------
----------------Without Sunda Shelf --------------------------

### Uniform senario coral records

In [7]:
jack_knife_result =np.zeros([60000,3])
site_name = ['Tahiti',"Barbados",'Sunda Shelf','Hydrographer','Noggin Pass',
            'Scotland']
for jack in range(6):
    
    all_uni_result = [Tahiti_uni_dis,Barbados_uni_dis,Sunda_uni_dis,NOG_uni_dis,
                           HYD_uni_dis,Scot_uni_dis]
    total_uni_std = [Tahiti_uni_std,Barbados_uni_std,Sunda_uni_std,HYD_uni_std,NOG_uni_std,
                         Scot_uni_std]
    A_index = ~(np.arange(0,6)==jack)
    A_jack = A[A_index]

    del all_uni_result[jack] #remove one site's observation
    del total_uni_std[jack] #remove one site's standard error 
    uni_test_result = []
    all_uni_result = np.array(all_uni_result)
    total_uni_std = np.array(total_uni_std)
    while len(uni_test_result)<10000:
        
        i = np.random.randint(0,20000,1)[0]
        obs = np.array([Tahiti_uni_dis.iloc[i],Barbados_uni_dis.iloc[i],Sunda_uni_dis.iloc[i],HYD_uni_dis.iloc[i],
                       NOG_uni_dis.iloc[i],Scot_uni_dis.iloc[i]])
        obs = obs[A_index] 
        opti = list(nnls(np.sqrt(1/np.array(total_uni_std)**2)[:,None] * A_jack,np.sqrt(1/np.array(total_uni_std)**2)*obs)[0])
        uni_test_result.append(opti)
       
    
    uni_test_result= np.array(uni_test_result)
    jack_knife_result[jack*10000:(jack+1)*10000,:]=uni_test_result
    print('----------------Without {:} -------------------------------------'.format(site_name[jack]))
    print('----------------MWP1A Magnitude & Sources with Uniform Distribution -----------')
    print('----------------Mean Result [95% confidence interval] (1 std)-----------------')
    print('North American Ice Sheet: {0:5.3f} [{1:5.3f},{2:5.3f}]  ({3:5.3f})'.format(np.mean(uni_test_result[:,0]),find_maximum_prob(uni_test_result[:,0],95)[0],find_maximum_prob(uni_test_result[:,0],95)[1],
                                                                                   np.std(uni_result[:,0])))
    print('West Antarctic Ice Sheet: {0:5.3f} [{1:5.3f},{2:5.3f}] ({3:5.3f})'.format(np.mean(uni_test_result[:,1]),find_maximum_prob(uni_test_result[:,1],95)[0],find_maximum_prob(uni_test_result[:,1],95)[1],
                                                                                  np.std(uni_test_result[:,1])))
    print('Scandinavian Ice Sheet:   {0:5.3f} [{1:5.3f},{2:5.3f}]  ({3:5.3f})'.format(np.mean(uni_test_result[:,2]),find_maximum_prob(uni_test_result[:,2],95)[0],find_maximum_prob(uni_test_result[:,2],95)[1],
                                                                                  np.std(uni_test_result[:,2])))
    print('----------------------------------------------------------------------')

print('----------------Overall Jackknife/Original Results --------------------')
print('NAIS: {0:5.3f}, {1:5.3f}  Bias: {2:5.3f} Bias Corrected: {3:5.3f} m'.format(np.mean(jack_knife_result[:,0]),np.mean(uni_result[:,0]),
                                                                           np.mean(jack_knife_result[:,0])-np.mean(uni_result[:,0]),
                                                                            np.mean(uni_result[:,0])-(np.mean(jack_knife_result[:,0])-np.mean(uni_result[:,0]))))
print('AIS: {0:5.3f}, {1:5.3f}  Bias: {2:5.3f} Bias Corrected: {3:5.3f} m'.format(np.mean(jack_knife_result[:,1]),np.mean(uni_result[:,1]),
                                                                           np.mean(jack_knife_result[:,1])-np.mean(uni_result[:,1]),
                                                                           np.mean(uni_result[:,1])-(np.mean(jack_knife_result[:,1])-np.mean(uni_result[:,1]))))
print('SIS:   {0:5.3f}, {1:5.3f}  Bias: {2:5.3f} Bias Corrected: {3:5.3f} m'.format(np.mean(jack_knife_result[:,2]),np.mean(uni_result[:,2]),
                                                                          np.mean(jack_knife_result[:,2])-np.mean(uni_result[:,2]),
                                                                        np.mean(uni_result[:,2])-( np.mean(jack_knife_result[:,2])-np.mean(uni_result[:,2]))))
print('----------------------------------------------------------------------')

----------------Without Tahiti -------------------------------------
----------------MWP1A Magnitude & Sources with Uniform Distribution -----------
----------------Mean Result [95% confidence interval] (1 std)-----------------
North American Ice Sheet: 10.562 [0.000,18.263]  (4.184)
West Antarctic Ice Sheet: 3.318 [0.000,11.440] (4.276)
Scandinavian Ice Sheet:   3.221 [0.000,5.981]  (1.887)
----------------------------------------------------------------------
----------------Without Barbados -------------------------------------
----------------MWP1A Magnitude & Sources with Uniform Distribution -----------
----------------Mean Result [95% confidence interval] (1 std)-----------------
North American Ice Sheet: 8.714 [0.000,17.220]  (4.184)
West Antarctic Ice Sheet: 5.547 [0.000,15.232] (5.946)
Scandinavian Ice Sheet:   4.379 [0.000,8.760]  (2.622)
----------------------------------------------------------------------
----------------Without Sunda Shelf -------------------------------

# 4. MWP-1A Sources Inversion with sea-level oscillation limit

### Empirical senario for coral records

In [8]:
emp_result = []
total_emp_std = np.array([Tahiti_emp_std,Barbados_emp_std,Sunda_emp_std,HYD_emp_std,NOG_emp_std,
                        Scot_emp_std]) 
while len(emp_result)<20000:
    i = np.random.randint(0,20000,1)[0]
    obs = np.array([Tahiti_emp_dis.iloc[i],Barbados_emp_dis.iloc[i],Sunda_emp_dis.iloc[i],HYD_emp_dis.iloc[i],
                   NOG_emp_dis.iloc[i],Scot_emp_dis.iloc[i]]) #compile all sample together
    opti = list(nnls(np.sqrt(1/np.array(total_emp_std)**2)[:,None] * A,np.sqrt(1/np.array(total_emp_std)**2)*obs)[0]) #non-negative least square
    if (opti[0]*0.75+opti[1]*1.09-opti[2]*0.74)<9: # filter out the results producing a sea-level oscillation
        emp_result.append(opti)
    
emp_result= np.array(emp_result)
print('----------------MWP1A Magnitude & Sources with Empirical Distribution -----------')
print('----------------Mean Result (95% confidence interval)----------------')
print('NAIS: {0:5.3f} [{1:5.3f},{2:5.3f}]'.format(np.mean(emp_result[:,0]),*find_maximum_prob(emp_result[:,0],95)[:2]))
print('AIS:  {0:5.3f} [{1:5.3f},{2:5.3f}]'.format(np.mean(emp_result[:,1]),*find_maximum_prob(emp_result[:,1],95)[:2]))
print('SIS:  {0:5.3f} [{1:5.3f},{2:5.3f}]'.format(np.mean(emp_result[:,2]),*find_maximum_prob(emp_result[:,2],95)[:2]))
print('Total MWP-1A Magnitude:    {0:5.3f} [{1:5.3f},{2:5.3f}]'.format(np.mean(np.sum(emp_result,axis=1)),*find_maximum_prob(np.sum(emp_result,axis=1),95)[:2]))

----------------MWP1A Magnitude & Sources with Empirical Distribution -----------
----------------Mean Result (95% confidence interval)----------------
NAIS: 11.283 [0.000,15.666]
AIS:  2.066 [0.000,9.422]
SIS:  4.939 [2.859,7.595]
Total MWP-1A Magnitude:    18.289 [14.624,21.957]


### Uniform senario coral records

In [9]:
uni_result = []
total_uni_std = np.array([Tahiti_uni_std,Barbados_uni_std,Sunda_uni_std,HYD_uni_std,NOG_uni_std,
                        Scot_uni_std])
while len(uni_result)<20000:
    i = np.random.randint(0,20000,1)[0]
    obs = np.array([Tahiti_uni_dis.iloc[i],Barbados_uni_dis.iloc[i],Sunda_uni_dis.iloc[i],HYD_uni_dis.iloc[i],
                   NOG_uni_dis.iloc[i],Scot_uni_dis.iloc[i]]) #compile all sample together
    opti = list(nnls(np.sqrt(1/np.array(total_uni_std)**2)[:,None] * A,np.sqrt(1/np.array(total_uni_std)**2)*obs)[0]) #non-negative least square
    if (opti[0]*0.75+opti[1]*1.09-opti[2]*0.74)<9: # filter out the results producing a sea-level oscillation

        uni_result.append(opti)
    
uni_result= np.array(uni_result)
print('----------------MWP1A Magnitude & Sources with Uniform Distribution -----------')
print('----------------Mean Result (95% confidence interval)----------------')
print('NAIS: {0:5.3f} [{1:5.3f},{2:5.3f}]'.format(np.mean(uni_result[:,0]),*find_maximum_prob(uni_result[:,0],95)[:2]))
print('AIS:  {0:5.3f} [{1:5.3f},{2:5.3f}]'.format(np.mean(uni_result[:,1]),*find_maximum_prob(uni_result[:,1],95)[:2]))
print('SIS:  {0:5.3f} [{1:5.3f},{2:5.3f}]'.format(np.mean(uni_result[:,2]),*find_maximum_prob(uni_result[:,2],95)[:2]))
print('Total MWP-1A Magnitude: {0:5.3f} [{1:5.3f},{2:5.3f}]'.format(np.median(np.sum(uni_result,axis=1)),*find_maximum_prob(np.sum(uni_result,axis=1),95)[:2]))

----------------MWP1A Magnitude & Sources with Uniform Distribution -----------
----------------Mean Result (95% confidence interval)----------------
NAIS: 11.493 [3.355,16.077]
AIS:  1.709 [0.000,7.373]
SIS:  4.561 [2.580,6.578]
Total MWP-1A Magnitude: 17.751 [15.078,20.412]


### Averaged results

In [10]:
print('----------------Averaged MWP1A Magnitude & Sources -----------')
print('----------------Median Result (95% confidence interval)----------------')
print('NAIS: {0:5.3f} [{1:5.3f},{2:5.3f}]'.format(np.median((uni_result[:,0]+emp_result[:,0])/2),*find_maximum_prob((uni_result[:,0]+emp_result[:,0])/2,95)[:2]))
print('AIS:  {0:5.3f} [{1:5.3f},{2:5.3f}]'.format(np.median((uni_result[:,1]+emp_result[:,1])/2),*find_maximum_prob((uni_result[:,1]+emp_result[:,1])/2,95)[:2]))
print('SIS:  {0:5.3f} [{1:5.3f},{2:5.3f}]'.format(np.median((uni_result[:,2]+emp_result[:,2])/2),*find_maximum_prob((uni_result[:,2]+emp_result[:,2])/2,95)[:2]))
print('Total MWP-1A Magnitude: {0:5.3f} [{1:5.3f},{2:5.3f}]'.format(np.median((np.sum(uni_result,axis=1)+np.sum(emp_result,axis=1))/2),*find_maximum_prob(np.sum(uni_result,axis=1),95)[:2]))

----------------Averaged MWP1A Magnitude & Sources -----------
----------------Median Result (95% confidence interval)----------------
NAIS: 12.107 [5.752,15.478]
AIS:  1.283 [0.000,5.872]
SIS:  4.723 [3.190,6.315]
Total MWP-1A Magnitude: 18.039 [15.078,20.412]


## Citation



## Final notes:

- Due to the different ramdom seeds, this code can possibly result in a slightly different results comparing to the results shown in our manuscript
- If you have any questions, feedbacks or comments, please feel free to contact [the corresponding author](yc-lin.com): yucheng.lin@durham.ac.uk
